
---
<big><big><big><big><big><big>Sieci neuronowe 2018</big></big></big></big></big></big>

---





---
<big><big><big><big><big>Self Organizing Maps SOM</big></big></big></big></big>

---



---

<id=tocheading><big><big><big><big>Spis treści</big></big></big></big>
<div id="toc"></div>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="nn_figures/" width="100%">

# Reguła Hebba
> Jeśli akson neuronu A jest wystarczająco blisko neuronu B i regularnie bierze udział w jego aktywacji, wtedy następuje pewien proces wzrostu lub zmian metabolicznych w obu komórkach tak, że zdolność A do aktywacji B wzrasta (Donald Hebb, 1949)

1. prosta interpretacja (McKay)
  * niech sygnał $m$ (zapach banana) powoduje wzrost aktywacji neuronu $m$
  * niech sygnał $n$ (żółty kształt) będzie związany z aktywacją neuronu $n$
  * jeśli te dwa sygnały będą występowały jednocześnie, to reguła __uczenia hebbowskiego__ spowoduje wzrost wag $w_{mn}$ i $w_{nm}$
  * jeśli później pojawi się jeden z tych sygnałów, to wywoła aktywację drugiego neuronu
2. to jest uczenie __asocjacyjne__
  $$\Delta w_{ij}=\eta x_i() y_j()$$
  * jednoczesna aktywacja powoduje wzrost
  * aktywacje asynchroniczne będą powodowały spadek
  * wciąż powtarzające się aktywacje będą powodowały saturację
    * waga przestaje się uczyć
    * potrzebne ograniczenia
3. to proces __samo-wzmocnienia__
  * uczenie __nienadzorowane__
4. __ograniczona__ liczba zasobów prowadzi do __współzawodnictwa__
  * pomiędzy neuronami bądź grupami neuronów
  * wybierane są te szybciej rosnące
  * neurony nie aktywowane zanikają
    * wraz z nie używanymi wagami
  * powstaje __zwycięzca__ w procesie __winner-takes-all__
    * zwycięzca staje się __detektorem cech__
5. neurony będące w bliskości do siebie __kooperują__
  * aktywowany neuron wpływa na aktywację neuronów ze sobą powiązanych
  * tworzą się obszary aktywacji
  * obszary aktywacji są grupami detektorów
  * kooperacja jest związana ze współzawodnictwem
6. __samo-organizacja__
  * kooperacja tworzy grupy aktywacji
  * rozdzielne grupy aktywacji tworzą strukturę systemu
7. w mózgu ssaków tworzą się obszary _Brodmana_ 
  * charakteryzujące się różną budową histopatologiczną
  * odpowiedzialne za różne funkcje

<img src="nn_figures/juliabuntainecerebralmap.jpg" width="75%" >

## Mapy Kohonena
<img src="nn_figures/som.png" width="55%" >
1. organizacja
  * neurony zwykle na 2-wymiarowej kwadratowej lub heksagonalnej siatce
  * każdy neuron ma __sąsiadów__ w odległości $1$ na siatce
2. każdy neuron ma związany wektor $w\in\mathbb{R}^n$, gdzie $n$ jest wymiarem wejścia
  * dla wejścia $x$, każdy neuron oblicza $w_k^Tx$
  * wszystkie $w_k$ mają taką samą długość
3. __współzawodnictwo__ zwycięzcą jest neuron o największej wartości iloczynu skalarnego
  * maksymalizacja $w^Tx$ jest równoważna __minimalizacji odległości__ $d(x,w_k)$
  $$i(x)=\underset{j}{\arg\,\min}\|x-w_j\|$$
  * wybór zwycięzcy odpowiada rzutowaniu ciągłej przestrzeni wejść na dyskretną mapę SOM
4. __kooperacja__ zwycięzca i jego sąsiedzi będą współpracowały: ich wagi będą modyfikowane w tym samym kierunku
<img src="nn_figures/som-learn.png" width="75%" >
  * __otoczenie__ $h_{ji(x)}$ dla zwycięzcy $i()$ jest wycentrowane na zwycięzcy i malejące 
    * gausowskie, trójkątne, etc.
    * mapa heksagonalna lepiej interpretuje odległość
    * w otoczeniu _odległość_ jest liczona __na siatce__, __nie w przestrzeni wejściowej__
    $$\begin{align}
    h_{ji(x)}(t)&=\exp\left(-\frac{d_{ji(x)}^2}{2\sigma^2(t)}\right)\\
    \sigma(t)&=\sigma(0)\exp\left(-\frac{t}{\tau}\right)
    \end{align}$$
      gdzie $t$ to krok, $\tau$ stała początkowa
      * zwykle sąsiedztwo obejmuje na początku całą mapę
5. __adaptacja__ zwycięzca i jego sąsiedzi będą w kolejnych krokach aktywaniej reagowały na podobny sygnał
  * uczenie jest hebbowskie
    * prowadzi do saturacji, więc potrzebne są ograniczenia - czynnik __zapominania__
    $$\Delta w_j=\eta y_j x - g(y_j) w_j$$
    * pierwszy składnik jest uczeniem hebbowskim
    * niech $y_j=h_{j i(x)}$
      * dla zwycięzcy $y_{i(x)}=0$
    * stąd pełne nauczanie
    $$\begin{align}
    \Delta w_j&=\eta(t) y_j x - g(y_j) w_j\\
    &=\eta(t) y_j (x - w_j)
    \end{align}$$
    dla $\eta(t)=\eta(0)\exp\left(-\frac{t}{\nu}\right)$
    * jest cały zestaw hipeparametrów: $\eta(t), \sigma(t)$
6. __samo-organizacja__ zwycięzca wraz z sąsiadami będą tworzyć grupę
7. w trakcie uczenia SOM przechodzi kilka etapów
  * wagi są początkowo losowane
  * początkowo nie ma więc żadnej organizacji
  * pierwszym etapem jest faza __wyprostowywania__
    * zbyt duża prędkość może uniemożliwić nawet wyprostowanie mapy
    * staje się on niemożliwy, gdy sąsiedztwo jest zbyt małe
  * w tym momencie sieć obejmuje tylko mały fragment wejścia
  * teraz jest etap __rozciągania__ do całkowitej reprezentacji
  * w końcu etap szczegółowego __dopasowywania__ do rozkładu

## Koder i dekoder
1. __Koder__ mapuje przestrzeń wejściową na dyskretną mapę SOM
2. __dekoder__ odtwarza kody mapy SOM z powrotem
  * wagi neuronu są zdeterminowane przez powtarzające się wejścia
  * odpowiadają centrom klastrów
  * tworzą dość dobrą aproksymację przestrzeni wejściowej
3. mapa SOM jest __topologicznie uporządkowana__
  * podobne wejścia są mapowane na ten sam lub nieodległe na siatce neurony
  * neurony blisko na mapie mają podobne wagi
  * grupy nieodległych neuronów można etykietować podobnymi cechami
4. SOM powinna dobrze przybliżać rozkład prawdopodobieństwa przestrzeni wejściowej
  * oszacowania mówią, że liczba neuronów odpowiadających małemu obszarowi jest proporcjonalna do $p_X^{1/x}$ do $p_X^{2/3}$
    * małe obszary wejścia są nadreprezentowane
    * duże są niedoreprezentowane
    
<img src="nn_figures/som-movies.png" width="105%" >
[mapa SOM dla 10 tysięcy stron webowych związanych z rozrywką]

## Parametry
1. wybór wymiarów siatki jest trudny
  * zbyt dokładna powoduje długi czas uczenia
  * zbyt rzadka nie da dokładności
2. możliwa jest adaptacja
  * rozpoczynamy od stosunkowo małej siatki
  * uczenie przechodzi fazę rozprostowywania
  * w każdym węźle zwycięzcy przechowywane są statystyki __błędów__ $$\sum\|x-w_{i(x)}\|$$
  * gdy sumaryczny błąd wszystkich węzłów przekroczy ustalony próg
    * wyszukiwane są wiersz/kolumny o największych sumarycznych błędach
      * będą to te dla których błąd był duży oraz były często zwycięzcami
    * __pomiędzy__ dwa wiersze/kolumny wstawiany jest __nowy__ wiersz/kolumna
      * wagi inicjowane są na średnie
      * błędy są dzielone przez $2$

# Demo
Bardzo fajne demo autora GNG [Bernda Fritzke](https://www.demogng.de/js/demogng.html?model=SOM&showAutoRestart&autoRestartTime=8)

# Growing Neural Gas
1. siatka rozpoczyna od dwóch neuronów połączonych krawędzią
  * krawędź ma parametr __wieku__
2. zostaje wylosowany $x\sim P_X$
  * neurony zostają posortowane według rosnącej __odległości__ od $x$
    * dla GNG __bez "growing"__ neuronów jest od razu N (niepołączonych) i są adaptowane wszystkie w coraz mniejszym stopniu
  * wszystkie krawędzie są postarzone o $1$
  * u zwycięzcy aktualizowany jest błąd $$err(s_1)=err(s_1)+\|w_1-x\|^2$$
  * dla wszystkich węzłów połączonych z $s_1$ następuje aktualizacja
    $$\begin{align}
    \Delta w_1&=\epsilon_1(x-w_1)\\
    \Delta w_j&=\epsilon_j(x-w_j)
    \end{align}$$
  * dla pierwszego $s_1$ i drugiego najbliższego węzła $s_2$
    * jeśli __nie są__ połączone krawędzią, to jest ona tworzona
    * wiek krawędzi $e(s_1, s_2)$ jest zerowany
  * wszystkie krawędzie o wieku większym od maksymalnego są __usuwane__
    * jeśli powoduje to utworzenie izolowanych węzłów, to są one także usuwane
3. co ustaloną liczbę kroków
  * wyszukiwany jest węzeł o największym zakumulowanym błędzie $s_f$
  * wśród jego sąsiadów wyszukiwany jest ten o największym błędzie $s_s$
  * pomiedzy te dwa wstawwiany jest nowy węzeł $$s_n=(s_f+s_s)/2$$
    * wstawiane są krawędzie $e(s_f,s_n)$ i $e(s_n,s_s)$
    * usuwana krawędź $e(s_f,s_s)$
  * błąd $s_f$ i $s_s$ jest zmniejszany przez podzielenie przez stałą
4. uczenie trwa aż do osiągnięcia maksymalnej liczby węzłów oraz stabilizacji błędu
5. rzeczywiście dynamiczny algorytm
  * działa w dynamicznym środowisku
  * rzeczywiście uczy się topologii
    * lepsze niż SOM Kohonena przybliżanie rozkładów
  * warunki stopu mogą być czasem trudne do łatwego ustalenia
  * może być czasochłonna
6. problemem zarówno SOM jak i GNG jest trudność z dopasowywaniem rozkładów na brzegu siatki
  * można spróbować torus
6. Bardzo dobra [implementacja](https://github.com/kudkudak/Growing-Neural-Gas)